- Прочитайте главы 12-14 из книги [Spark: The Definitive Guide](https://analyticsdata24.files.wordpress.com/2020/02/spark-the-definitive-guide40www.bigdatabugs.com_.pdf#%5B%7B%22num%22%3A484%2C%22gen%22%3A0%7D%2C%7B%22name%22%3A%22XYZ%22%7D%2C72%2C792%2Cnull%5D).
- Прочитайте главу 2 из книги [Hadoop: The Definitive Guide](https://grut-computing.com/HadoopBook.pdf#%5B%7B%22num%22%3A348%2C%22gen%22%3A0%7D%2C%7B%22name%22%3A%22XYZ%22%7D%2Cnull%2C588.58502%2Cnull%5D).
- Выполните задания.

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Lab4").getOrCreate()
sc=spark.sparkContext

textFiles = sc.textFile('hdfs://localhost:9000//user/student/gutenberg/*')

textFiles.first()

№0: Откройте каталог "/home/student/Work/Labs/3.Hadoop/WordCount" и изучите его содержимое.

Здесь представлена реализация подсчета слов в текстовых документах по парадигме MapReduce для Apache Hadoop. В папке Input находятся текстовые файлы с входными данными. Они же загружены в файловую систему HDFS по адресу hdfs://localhost:9000//user/student/gutenberg. В папке Src представлен исходный код стадий map и reduce. В папке Scripts есть два скрипта: cluster_run.sh и local_run.sh. Первый выполняет подсчет слов на платформе Hadoop. Второй выполняет ту же задачу без Hadoop. В папку Output попадают результаты выполнения скриптов.

Код ниже подсчитывает количество вхождений слов в текстовые документы с помощью платформы Apache Spark. Сравните этот код с реализацией той же задачи для Apache Hadoop.

In [ ]:
words = textFiles\
    .flatMap(lambda line: line.split(" "))

counts = words\
    .map(lambda word: (word, 1)) \
    .reduceByKey(lambda x, y: x + y)

counts.take(6)

№1: Найдите количество вхождений только тех слов, длина которых не менее 5 символов. Выведите их в порядке убывания.

[('which', 3637),
 ('their', 1691),
 ('there', 1224),
 ('other', 954),
 ('would', 884),
 ('these', 846)]

№2: Найдите количество повторений последовательностей из двух слов в текстах. Выведите их в порядке убывания.

[(('of', 'the'), 6900),
 (('in', 'the'), 3501),
 (('to', 'the'), 2208),
 (('and', 'the'), 1637),
 (('on', 'the'), 1618),
 (('from', 'the'), 1231)]

In [ ]:
bigrams = textFiles.flatMap(lambda line: line.split("."))\
   .map(lambda line: line.strip().split(" "))\
   .flatMap(lambda xs: (tuple(x) for x in zip(xs, xs[1: ])))

pair_counts = bigrams\
    ...

№3: Найдите самое длинное слово из тех, которые состоят только из буквенно-цифровых символов.

№4: Для каждой буквы найдите количество уникальных слов в текстах, которые на нее начинаются. Приведите буквы к верхнему регистру, отсортируйте их в алфавитном порядке.

[('A', 4268), ('B', 4460), ('C', 6660), ('D', 4156), ('E', 2856), ('F', 3482)]

In [ ]:
unique_counts = words\
    ...

№ 5: Откройте каталог "/home/student/Work/Labs/3.Hadoop/Weather" и изучите его содержимое. Здесь представлен код MapReduce для Apache Hadoop, который обрабатывает погодные данные [National Climatic Data Center](http://www.ncdc.noaa.gov/). Подробности про задачу, которую решает данная программа, прочитайте во второй главе [Hadoop: The Definitive Guide](https://grut-computing.com/HadoopBook.pdf).

Погодные данные загружены в файловую систему HDFS по адресу hdfs://localhost:9000/user/student/weather. Там присутствуют данные только за 2022 год. Реализуйте решение той же задачи с погодными данными в Apache Spark с помощью методов flatMap и reduceByKey.

In [ ]:
import re
import sys

weatherFiles = sc.wholeTextFiles('hdfs://localhost:9000/user/student/weather/*')

def mapper(text):
    ...

def reducer(a, b):
    ...
    
weatherFiles\
    .flatMap(mapper)\
    .reduceByKey(reducer)\
    .collect()

№6: Решите задачу из задания 5 с помощью SQL и методов DataFrame.

In [ ]:
from pyspark.sql.functions import col

weatherLines = sc.textFile('hdfs://localhost:9000/user/student/weather/*')
weatherColumns = ["year","temperature", "quality"]
weatherDF = weatherLines\
    .map(lambda val: (int(val[15:19]), int(val[87:92]), int(val[92:93])))\
    .toDF(weatherColumns)

weatherDF.createOrReplaceTempView("Weather")

sqlWay = spark.sql("""

""")

dataFrameWay = weatherDF\
    ...

print(sqlWay.rdd.map(lambda x: (x[0], x[1])).collect())
print(dataFrameWay.rdd.map(lambda x: (x[0], x[1])).collect())